In [ ]:
# OVERALL CODE FOR RUNNING EVERYTHING

from encoder import *
from pydub import AudioSegment
from pydub.playback import play
from PIL import Image
from IPython.display import display

INPUT_AUDIO = "obiwan.wav"
INPUT_IMAGE = "babyyoda.png"

# read (and play) first audio file:
play(AudioSegment.from_wav(INPUT_AUDIO))

# get audio bits
audio_bitstream, a_bitstream = get_audio_bits(audio_file)

# display image for encoding
img = Image.open(INPUT_IMAGE)
display(img)

#---------------------------------------------------------------------------------------------------------------------------------------
# Image processing ---------------------------------------------------------------------------------------------------------------------
# -> Outputs binary_pixels
# binary_pixels has the original image data in binary and in rgb format 

t_image0 = time.time()

# Load the PNG image
image = Image.open(INPUT_IMAGE)

# Get the size of the image
width, height = image.size

# Checks image to ensure correct size
if width != 1000 or height != 1000:       
    print("Wrong image size!")
else:
    print("Image size is correct!\nStarting encryption...")

    # Convert the image to RGB mode 
    image_rgb = image.convert("RGB")

    # Get pixel data of image
    pixels = image_rgb.load()

    # Convert RGB values to binary and store in a 2D array
    binary_pixels = []
    for y in range(image.height):
        row = []
        for x in range(image.width):
            r, g, b = pixels[x, y]
            binary_r = bin(r)[2:].zfill(8)  # Convert red value to binary with 8 bits
            binary_g = bin(g)[2:].zfill(8)  # Convert green value to binary with 8 bits
            binary_b = bin(b)[2:].zfill(8)  # Convert blue value to binary with 8 bits
            row.append((binary_r, binary_g, binary_b))
        binary_pixels.append(row)

t_image1 = time.time()

total_image = t_image1 - t_image0
        
# Access original pixel (1, 1) data for testing
print("Original image pixel information in binary for pixel (1, 1)")
print(binary_pixels[0][0])

#----------------------------------------------------------------------------------------------------------------------------------------
# Image Encoding ------------------------------------------------------------------------------------------------------------------------
# -> outputs encoded_bitstream
# encoded_bitstream has the encoded image data in a binary bitstream 
# Every 24 bits of encoded_bitstream represents the information of 1 pixel
# Stored using big endian 

t_imagee0 = time.time()

# Convert binary pixel data back to image and encode the image simultaneously
encoded_image = Image.new("RGB", (width, height))
encoded_pixels = encoded_image.load()

# Initialise variables to store the continuous bit stream
encoded_bitstream = "" # Will contain the encoded image bitstream for sending
stream_index = 0
encoded = [] # Contains the encoded image in a 2D array representing the RGB colours of the image 

# Iterates through each pixel in the image
for y in range(image.height):
    row = []
    for x in range(image.width):
        # Get the RGB values of the current pixel
        binary_r, binary_g, binary_b = binary_pixels[y][x]

        # Modifies last two digits of each color component if audio stream has bits left
        if stream_index < len(audio_bitstream) - 1:
            r = binary_r[:-2] + audio_bitstream[stream_index] + audio_bitstream[stream_index + 1]
            stream_index += 2
        else:
            r = binary_r
            
        if stream_index < len(audio_bitstream) - 1:
            g = binary_g[:-2] + audio_bitstream[stream_index] + audio_bitstream[stream_index + 1]
            stream_index += 2
        else:
            g = binary_g
            
        if stream_index < len(audio_bitstream) - 1:
            b = binary_b[:-2] + audio_bitstream[stream_index] + audio_bitstream[stream_index + 1]
            stream_index += 2
        else:
            b = binary_b

        # Store modified RGB values in the encoded array
        row.append((r, g, b))

        # Append the modified bits to the continuous bit stream
        encoded_bitstream += r + g + b

        # Convert binary RGB values back to integer
        r_int = int(r, 2)
        g_int = int(g, 2)
        b_int = int(b, 2)
        
        # Set pixel value in the new image
        encoded_pixels[x, y] = (r_int, g_int, b_int)

    encoded.append(row)

# Output the first few bits of the continuous bit stream
print("First few bits of encoded image bit stream:")
print(encoded_bitstream[:24])

#print("first encoded bits: ", encoded[:1] )

# Save the encoded image
encoded_image.save("encoded_image.png")  
print("Done!")

# Saving to a txt file ------------------------------------------------------------------------------------------------------------------------
# Save the encoded bit stream to a file
with open("encoded_bitstream.txt", "w") as file:
    for i in range(0, len(encoded_bitstream), 8):
        byte = encoded_bitstream[i:i+8]  # Extracting 8 bits (1 byte)
        file.write(byte + "\n")

t_imagee1 = time.time()

total_imagee = t_imagee1 - t_imagee0

print("Total time for audio processing: ", total_audio, " seconds")
print("Total time for image processing: ", total_image, " seconds")
print("Total time for image encoding: ", total_imagee, " seconds")